Based on coordinates, decide if location is convenient or inconvenient

In [5]:
import osmnx as ox
import pandas as pd
import time
import glob

In [32]:
def auto_rename_columns(df):
    long_list = ["Longitude", "Lng"]
    for col_name in long_list:
        if col_name in df.columns:
            df.rename(columns={col_name: 'long'}, inplace=True)
            break

    lat_list = ["Latitude", "Lat"]
    for col_name in lat_list:
        if col_name in df.columns:
            df.rename(columns={col_name: 'lat'}, inplace=True)
            break

    postcode_list = ["Postcode", "POSTCODE"]
    for col_name in postcode_list:
        if col_name in df.columns:
            df.rename(columns={col_name: 'postcode'}, inplace=True)
            break

    price_list = ["PRODUCT_PRICE", "Price"]
    for col_name in price_list:
        if col_name in df.columns:
            df.rename(columns={col_name: 'price'}, inplace=True)
            break

    if "FuelCode" in df.columns:
        df.rename(columns={"FuelCode": 'fueltype'}, inplace=True)

    brand_list = ["BrandName", "BRAND_DESCRIPTION"]
    for col_name in brand_list:
        if col_name in df.columns:
            df.rename(columns={col_name: 'brand'}, inplace=True)
            break

    if "PUBLISH_DATE" in df.columns:
        df.rename(columns={"PUBLISH_DATE": 'collection_date'}, inplace=True)

    if "Address" in df.columns:
        df.rename(columns={"Address": 'address'}, inplace=True)

    return df


def combine_csv(path):
    # Get all csv files in the folder
    all_files = glob.glob(path + "/*.csv")

    # Create an empty list to add the dataframes to
    df_list = []

    # Loop through the list of filepaths & read each one into a df
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)

    # Concatenate all dataframes in the list
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)

    # Return the dataframe
    return combined_df

def extract_address(state_fuel):
    address = state_fuel[["address", "lat", "long"]].drop_duplicates()
    return address


def is_convenient(lat, long):
    # check performance
    #start_time = time.time()
    
    point = (lat, long)
    # only include main road structures
    tags = {'highway': ['motorway', 'trunk', 'primary']}

    try:
        # features_from_point to get features within 500 meters
        features = ox.features_from_point(point, tags, dist=500)
        
        # check performance
        #end_time = time.time()
        #print(end_time - start_time)
        
        # if no features
        if features is None or features.empty:
            return False
        # if exist features
        else:
            return True
    except Exception as e:
        # handle features_from_point error
        return False
      

def mutate_convenient(state_fuel):
    address = extract_address(state_fuel)

    # Check if convenient
    address['convenient'] = address.apply(lambda row: is_convenient(row['lat'], row['long']), axis=1)

    # Left join to original dataframe
    state_fuel = pd.merge(state_fuel, address, on=['address', 'lat', 'long'], how='left')

    return state_fuel

In [12]:
# import nsw
# nsw = combine_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/nsw")
nt = combine_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/nt")
qld = combine_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/qld")
sa = combine_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/sa")
tas = combine_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/tas")

In [20]:
# rename dataframe
# nsw = auto_rename_columns(nsw)
nt = auto_rename_columns(nt)
qld = auto_rename_columns(qld)
sa = auto_rename_columns(sa)
tas = auto_rename_columns(tas)


In [ ]:
# nsw_address = extract_address(nsw)
nt_address = extract_address(nt)
qld_address = extract_address(qld)
sa_address = extract_address(sa)
tas_address = extract_address(tas)

In [29]:
# nsw_address["convenient"] = nsw_address.apply(lambda row: is_convenient(row["lat"], row["long"]), axis=1)
nt_address["convenient"] = nt_address.apply(lambda row: is_convenient(row["lat"], row["long"]), axis=1)
qld_address["convenient"] = qld_address.apply(lambda row: is_convenient(row["lat"], row["long"]), axis=1)
sa_address["convenient"] = sa_address.apply(lambda row: is_convenient(row["lat"], row["long"]), axis=1)
tas_address["convenient"] = tas_address.apply(lambda row: is_convenient(row["lat"], row["long"]), axis=1)


In [30]:
# export as csv
# nsw_address.to_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/nsw_address.csv", index=False)
nt_address.to_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/nt_address.csv", index=False)
qld_address.to_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/qld_address.csv", index=False)
sa_address.to_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/sa_address.csv", index=False)
tas_address.to_csv("D:/UOA WORK/Summer Research/Data_Journalism_ASRS/fuel-prices/tas_address.csv", index=False)
